<a href="https://colab.research.google.com/github/ShreyJ1729/EVA6-TSAI/blob/main/05-CodingDrillDown/Iteration_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.convblock1 = nn.Sequential(
            nn.Conv2d(1, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout2d(0.01),

            nn.Conv2d(10, 10, 3),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout2d(0.01),

            nn.Conv2d(10, 10, 3),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout2d(0.01),

            nn.MaxPool2d(2, 2)
        )

        self.convblock2 = nn.Sequential(
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout2d(0.01),

            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout2d(0.01),

            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
            nn.Dropout2d(0.01),

            nn.MaxPool2d(2, 2)
        )
        
        self.convblock3 = nn.Sequential(
            nn.Conv2d(10, 16, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout2d(0.01),

            nn.Conv2d(16, 15, 3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(15),
            nn.Dropout2d(0.01),
            
            nn.Conv2d(15, 10, 3, padding=1),
            nn.AvgPool2d(6)
        )

        self.fcblock = nn.Linear(10, 10)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = x.view(-1, 10)
        x = self.fcblock(x)
        return F.log_softmax(x, dim=1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
         Dropout2d-4           [-1, 10, 28, 28]               0
            Conv2d-5           [-1, 10, 26, 26]             910
              ReLU-6           [-1, 10, 26, 26]               0
       BatchNorm2d-7           [-1, 10, 26, 26]              20
         Dropout2d-8           [-1, 10, 26, 26]               0
            Conv2d-9           [-1, 10, 24, 24]             910
             ReLU-10           [-1, 10, 24, 24]               0
      BatchNorm2d-11           [-1, 10, 24, 24]              20
        Dropout2d-12           [-1, 10, 24, 24]               0
        MaxPool2d-13           [-1, 10, 12, 12]               0
           Conv2d-14           [-1, 10,

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm.notebook import tqdm
def train(model, device, train_loader, optimizer, scheduler, epoch):
    epoch_losses = []
    model.train()
    pbar = tqdm(train_loader, leave=False)
    correct = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        epoch_losses.append(loss.item())
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        tqdm.write(f'\r[Epoch {epoch}] loss={round(sum(epoch_losses)/len(epoch_losses), 4)}', end="")
    tqdm.write('\r[Epoch {}]\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(epoch, 
        round(sum(epoch_losses)/len(epoch_losses), 4), correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))
    scheduler.step()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    tqdm.write('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, 0.85)

for epoch in range(15):
    train(model, device, train_loader, optimizer, scheduler, epoch)
    test(model, device, test_loader)

[Epoch 0]
Train set: Average loss: 0.4822, Accuracy: 50249/60000 (83.75%)
Test set: Average loss: 0.0734, Accuracy: 9776/10000 (97.76%)



[Epoch 1]
Train set: Average loss: 0.0860, Accuracy: 58436/60000 (97.39%)
Test set: Average loss: 0.0515, Accuracy: 9838/10000 (98.38%)



[Epoch 2]
Train set: Average loss: 0.0615, Accuracy: 58914/60000 (98.19%)
Test set: Average loss: 0.0363, Accuracy: 9888/10000 (98.88%)



[Epoch 3]
Train set: Average loss: 0.0503, Accuracy: 59093/60000 (98.49%)
Test set: Average loss: 0.0312, Accuracy: 9902/10000 (99.02%)



[Epoch 4]
Train set: Average loss: 0.0423, Accuracy: 59230/60000 (98.72%)
Test set: Average loss: 0.0290, Accuracy: 9900/10000 (99.00%)



...

[Epoch 10]
Train set: Average loss: 0.0247, Accuracy: 59540/60000 (99.23%)
Test set: Average loss: 0.0202, Accuracy: 9936/10000 (99.36%)



[Epoch 11]
Train set: Average loss: 0.0236, Accuracy: 59565/60000 (99.28%)
Test set: Average loss: 0.0182, Accuracy: 9940/10000 (99.40%)



[Epoch 12]
Train set: Average loss: 0.0224, Accuracy: 59577/60000 (99.30%)
Test set: Average loss: 0.0176, Accuracy: 9948/10000 (99.48%)



[Epoch 13]
Train set: Average loss: 0.0225, Accuracy: 59579/60000 (99.30%)
Test set: Average loss: 0.0185, Accuracy: 9940/10000 (99.40%)



[Epoch 14]
Train set: Average loss: 0.0218, Accuracy: 59604/60000 (99.34%)
Test set: Average loss: 0.0184, Accuracy: 9943/10000 (99.43%)



Target:
- 99.4% Test Accuracy (sticks till the end)
- Under 10k parameters

Results:
- 99.4% accuracy (stuck till the end)
- ~10k parameters

Analysis:
- LR scheduler worked beautifully near the end